# Neural Archaeology: Decoding and Rewiring the Hidden Mind of Language Models

## ECE4424/CS4824: Machine Learning, Fall 2025
**Instructor**: Prof. Ming Jin

### Submission Requirements

1. **Complete ALL code implementations** marked with `# YOUR CODE HERE`
2. **Complete ALL analysis sections** marked as "Required Analysis"
3. **Run all cells** to show outputs (including visualizations)
4. **Save outputs**: Ensure all plots are visible in the notebook
5. **Submit  the PDF of the Notebook**: Print the Notebook with clear results in a PDF format.

**Note**: Required Analysis sections are clearly marked throughout the notebook.

## Project Overview: The Quest to Understand Machine Minds

<img src="assets/brain-neural.png" alt="Brain Neural Network" width="300" align="right" style="margin-left: 20px; margin-bottom: 10px;">

Imagine having the ability to peer directly into a human brain, watching thoughts form in real-time, identifying the exact neurons that fire when someone feels joy versus fear, and even being able to gently nudge those neurons to change the person's emotional state. This level of understanding and control has long been a dream in neuroscience. Today, we're going to achieve something remarkably similar, but with artificial intelligence.


Language models like GPT and LLaMA are often called "black boxes". We know what goes in and what comes out, but the middle remains mysterious. In this notebook, we'll embark on a journey of **neural archaeology**, excavating the hidden layers of these models to understand how they encode concepts like safety, emotion, and truthfulness.

This notebook builds on several groundbreaking research directions: **representation engineering** from [Zou et al. (2023)](https://arxiv.org/abs/2310.01405), the **circuit breakers** methodology from [Zou et al., 2024](https://www.circuit-breaker.ai/), and refusal mechanisms inspired by [Zhang et al., 2024](https://arxiv.org/abs/2409.14586) and [Sel et al., 2025](https://arxiv.org/abs/2503.08919). We synthesize these techniques into a hands-on exploration where you'll build practical safety detectors and discover how emotions organize geometrically in activation space.

By the end of this notebook, you'll have built a complete system that can:
1. **Read the model's mind**: extracting and visualizing its internal thought patterns
2. **Identify cognitive signatures**: finding the neural patterns for safety, emotions, and other concepts  
3. **Build safety detectors**: creating classifiers to identify harmful content
4. **Understand representation geometry**: exploring how concepts organize in neural space

This hands-on exploration connects cutting-edge research to practical implementation, giving you both theoretical understanding and engineering skills in AI safety and interpretability.

## Learning Objectives

Through this notebook, you will:
- **Implement PCA from scratch** to understand unsupervised dimensionality reduction
- **Apply K-means clustering** to discover natural groupings in neural representations
- **Compare supervised vs unsupervised learning** for representation discovery
- **Analyze layer-wise information processing** in Transformer architectures
- **Build and evaluate safety detection systems** using neural representations
- **Test adversarial robustness** of safety detection systems
- **Think critically about AI safety and ethics** through hands-on experimentation

## Setup Environment

### Quick Start Installation

In [ ]:
# ============================================================
# CORE IMPORTS
# ============================================================
import os
import torch
import torch.nn.functional as F
import numpy as np
import json
import pickle
import gc
import os
import sys
import subprocess
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

# Transformers and model utilities
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# Scientific computing
from sklearn.decomposition import PCA as SklearnPCA
from sklearn.metrics import silhouette_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# ============================================================
# ENVIRONMENT DETECTION AND CONFIGURATION
# ============================================================
IS_COLAB = 'google.colab' in sys.modules
IS_MPS = torch.backends.mps.is_available()

print(f"Environment: {'Google Colab' if IS_COLAB else 'Local'}")
print(f"MPS Available: {IS_MPS}")

# Device and parameter configuration
if IS_COLAB and torch.cuda.is_available():
    device = torch.device("cuda")
    DTYPE = torch.float16
    BATCH_SIZE = 16
elif IS_MPS:
    device = torch.device("mps")
    DTYPE = torch.float32  # MPS doesn't handle float16 well
    BATCH_SIZE = 8
else:
    device = torch.device("cpu")
    DTYPE = torch.float32
    BATCH_SIZE = 4

print(f"Neural Laboratory initialized on {device}")
print(f"PyTorch version: {torch.__version__}")
print(f"Data type: {DTYPE}")
print(f"Batch size: {BATCH_SIZE}")

# ============================================================
# REPRODUCIBILITY AND UTILITIES
# ============================================================
# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

def clear_memory():
    """Clear GPU/CPU memory caches to prevent OOM during long operations"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif IS_MPS:
        torch.mps.empty_cache()

def get_layer_ranges(model, print_config=True):
    """
    Get layer ranges for early, middle, and late layers based on model architecture.

    This function dynamically calculates appropriate layer indices based on the
    total number of layers in the model, making the code compatible with different
    architectures (e.g., SmolLM with 24 layers).

    Args:
        model: The transformer model
        print_config: Whether to print the layer configuration (default: True)

    Returns:
        dict: Dictionary containing:
            - 'total': Total number of layers
            - 'early': List of early layer indices (first ~25%)
            - 'middle': List of middle layer indices (middle ~50%)
            - 'late': List of late layer indices (last ~25%)
            - 'exploration': Sampled layers for exploration (8-10 layers)
            - 'safety_layers': Layers for safety detection (last 4 layers)
            - 'last_layer': Last layer index
    """
    n_layers = model.config.num_hidden_layers

    # Define ranges as proportions
    early_end = n_layers // 4
    middle_start = n_layers // 4
    middle_end = 3 * n_layers // 4
    late_start = 3 * n_layers // 4

    # Create explicit layer lists
    early_layers = list(range(0, early_end))
    middle_layers = list(range(middle_start, middle_end))
    late_layers = list(range(late_start, n_layers))

    # Sample layers for exploration (evenly spaced)
    exploration_layers = [0]  # Always include first layer
    step = max(1, n_layers // 8)
    exploration_layers.extend(list(range(step, n_layers, step)))
    if exploration_layers[-1] != n_layers - 1:
        exploration_layers.append(n_layers - 1)  # Always include last layer

    # Safety-critical layers (last 4 layers where abstract concepts emerge)
    safety_layers = list(range(max(0, n_layers - 4), n_layers))

    config = {
        'total': n_layers,
        'early': early_layers,
        'middle': middle_layers,
        'late': late_layers,
        'exploration': exploration_layers,
        'safety_layers': safety_layers,
        'last_layer': n_layers - 1
    }

    # Print configuration if requested
    if print_config:
        # Extract model name from config
        model_name = model.config._name_or_path.split('/')[-1] if hasattr(model.config, '_name_or_path') else 'model'

        print(f"\n{'='*60}")
        print(f"Layer Configuration for {model_name} ({n_layers} layers)")
        print(f"{'='*60}")

        # Early layers
        early_range = f"0-{early_end-1}" if early_layers else "none"
        early_preview = str(early_layers[:5]) + "..." if len(early_layers) > 5 else str(early_layers)
        print(f"Early layers ({early_range}):   {early_preview}")

        # Middle layers
        if middle_layers:
            middle_range = f"{middle_layers[0]}-{middle_layers[-1]}"
            middle_preview = str(middle_layers[:5]) + "..." if len(middle_layers) > 5 else str(middle_layers)
            print(f"Middle layers ({middle_range}): {middle_preview}")

        # Late layers
        late_range = f"{late_layers[0]}-{late_layers[-1]}" if late_layers else "none"
        print(f"Late layers ({late_range}):   {late_layers}")

        # Other info
        print(f"Exploration sample:    {exploration_layers}")
        print(f"Safety layers:         {safety_layers}")
        print(f"Last layer:            {config['last_layer']}")
        print(f"{'='*60}\n")

    return config

# ============================================================
# DATA AND ENVIRONMENT SETUP
# ============================================================
if IS_COLAB:
    # Colab setup
    !pip install torch transformers datasets scikit-learn matplotlib seaborn tqdm -q
    !pip install accelerate sentencepiece -q
    !git clone https://github.com/jinming99/learn-ml-by-building.git
    DATA_PATH = "/content/learn-ml-by-building/Project 2 Neural Archaeology/data"
    sys.path.append('/content/learn-ml-by-building/Project 2 Neural Archaeology')
else:
    # Local setup
    DATA_PATH = "./data"

    # Download data if it doesn't exist
    if not os.path.exists(DATA_PATH):
        print("Downloading data...")
        subprocess.run(["git", "clone", "https://github.com/jinming99/learn-ml-by-building.git", "temp_repo"])
        subprocess.run(["cp", "-r", "temp_repo/Project 2 Neural Archaeology/data", DATA_PATH])
        subprocess.run(["rm", "-rf", "temp_repo"])
        print("Data downloaded!")

# Create result directories
dirs_to_create = [
    'models',
    'results/visualizations',
    'results/evaluations',
    'results/saved_states',
    'results/student_analysis'
]

for dir_path in dirs_to_create:
    os.makedirs(dir_path, exist_ok=True)

print(f"Data path: {DATA_PATH}")
print("Setup complete!")

## Section 0: Foundation - Building Our Neural Laboratory

In neuroscience, before we can study the brain, we need proper instruments: EEG machines, fMRI scanners, and precise measurement tools. Similarly, we'll build our toolkit for exploring language models.

### Loading the Circuit Breaker Dataset: Our Moral Training Data

The [circuit breaker dataset](https://github.com/GraySwanAI/circuit-breakers) contains examples of safe and unsafe model behaviors. We'll use this to teach our model to recognize and avoid harmful outputs.


In [ ]:
def load_circuit_breaker_data(base_path=DATA_PATH):
    """
    Load circuit breaker training and validation data.

    This dataset contains the moral knowledge we'll implant into our model:
    - Prompts: Potentially unsafe user requests
    - Unsafe outputs: What the model shouldn't say
    - Safe outputs: Appropriate refusals
    """
    data = {}

    # Load from safety subdirectory
    safety_path = os.path.join(base_path, 'safety')

    # Load training data
    train_path = os.path.join(safety_path, 'circuit_breakers_train.json')
    if not os.path.exists(train_path):
        raise FileNotFoundError(f"Required file not found: {train_path}")

    with open(train_path, 'r') as f:
        train_data = json.load(f)

    # Load validation data
    val_path = os.path.join(safety_path, 'circuit_breakers_val.json')
    if not os.path.exists(val_path):
        raise FileNotFoundError(f"Required file not found: {val_path}")

    with open(val_path, 'r') as f:
        val_data = json.load(f)

    # Extract components
    data['train'] = {
        'prompts': [item['prompt'] for item in train_data],
        'unsafe_outputs': [item['output'] for item in train_data],
        'safe_outputs': [item['llama3_output'] for item in train_data],
        'categories': [item.get('category', 'unknown') for item in train_data]
    }

    data['val'] = {
        'prompts': [item['prompt'] for item in val_data],
        'unsafe_outputs': [item['output'] for item in val_data],
        'safe_outputs': [item['llama3_output'] for item in val_data],
        'categories': [item.get('category', 'unknown') for item in val_data]
    }

    print(f"Loaded {len(data['train']['prompts'])} training examples")
    print(f"Loaded {len(data['val']['prompts'])} validation examples")

    # Show category distribution
    categories = set(data['train']['categories'])
    print(f"Categories: {list(categories)[:5]}{'...' if len(categories) > 5 else ''}")

    return data

# Load the moral training data
cb_data = load_circuit_breaker_data()

# Examine a sample
print("\nSample Circuit Breaker Training Example:")
print("="*60)
idx = 0
print(f"User Prompt: {cb_data['train']['prompts'][idx]}")
print(f"Unsafe Response: {cb_data['train']['unsafe_outputs'][idx][:200]}...")
print(f"Safe Response: {cb_data['train']['safe_outputs'][idx][:200]}...")

### Loading Emotion Data: The Affective Palette

The emotion dataset contains scenarios that evoke different emotional responses. We'll use this to understand how the model represents emotions internally.


In [ ]:
def load_emotion_data(base_path=DATA_PATH):
    """
    Load emotion datasets - the building blocks of affective computing.

    Each emotion category contains scenarios that evoke that emotion.
    We'll use these to map the model's emotional landscape.
    """
    emotions = {}

    emotions_path = os.path.join(base_path, 'emotions')
    if not os.path.exists(emotions_path):
        raise FileNotFoundError(f"Emotions directory not found: {emotions_path}")

    emotion_files = [f for f in os.listdir(emotions_path) if f.endswith('.json')]

    if not emotion_files:
        raise FileNotFoundError(f"No emotion JSON files found in {emotions_path}")

    for filename in emotion_files:
        emotion_name = filename.replace('.json', '')
        filepath = os.path.join(emotions_path, filename)

        with open(filepath, 'r') as f:
            scenarios = json.load(f)
            emotions[emotion_name] = scenarios

    print(f"Loaded {len(emotions)} emotion categories:")
    for emotion, scenarios in emotions.items():
        print(f"  {emotion}: {len(scenarios)} scenarios")

    return emotions

# Load emotional training data
emotion_data = load_emotion_data()

# Sample the emotional palette
print("\nEmotional Palette Samples:")
print("="*60)
for emotion in list(emotion_data.keys()):
    print(f"\n{emotion.upper()}:")
    for scenario in emotion_data[emotion][:2]:
        print(f"  - {scenario}")

### Model Architecture: SmolLM-1.7B-Instruct

We'll work with **SmolLM-1.7B-Instruct**, a compact yet capable language model for our neural archaeology exploration:

- **Architecture**: 24 layers, 1.7B parameters
- **Usage**: Exploration and concept learning, understanding representation structure, training safety detectors
- **Rationale**: Fast iteration for learning, lower memory requirements, same architectural principles as larger models

This model is powerful enough to exhibit complex behaviors while remaining efficient to run on standard hardware.


In [ ]:
from huggingface_hub import login

# Replace "YOUR_HF_TOKEN" with the actual token you copied
login(token="YOUR_HF_TOKEN")

In [ ]:
def load_model_and_tokenizer(model_name="HuggingFaceTB/SmolLM-1.7B-Instruct"):
    """
    Load a language model for experimentation.

    Supports: SmolLM-1.7B-Instruct and other HuggingFace models.
    """
    print(f"Preparing model {model_name} for experimentation...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Use environment-specific settings
    if IS_COLAB:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=DTYPE,
            device_map="auto"
        )
    else:
        # For local, don't use device_map="auto" as it may not work well with MPS
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=DTYPE
        )
        model = model.to(device)  # Explicitly move to device

    # Ensure padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval()

    # Standard transformer architecture - access pattern: model.model.layers[i]
    layer_attr = "model.layers"

    print(f"Loaded {model_name}")
    print(f"  Hidden dim: {model.config.hidden_size}, Layers: {model.config.num_hidden_layers}")

    return model, tokenizer, layer_attr

model, tokenizer, layer_attr = load_model_and_tokenizer()

# Initialize layer ranges for SmolLM
LAYER_CONFIG = get_layer_ranges(model)

### Test the Model

Try different prompts to see how the model responds. The model uses chat templates for instruction following.

In [ ]:
def test_model_generation(prompt, max_new_tokens=100):
    """
    Test the model with a custom prompt.

    Args:
        prompt: Your question or instruction
        max_new_tokens: Maximum length of response
    """
    print(f"Model: SmolLM-1.7B-Instruct")
    print(f"Prompt: {prompt}")

    # Use chat template for instruct models
    messages = [{"role": "user", "content": prompt}]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Handle device placement properly
    if IS_COLAB:
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    else:
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.2, top_p=0.9, do_sample=True)

    # Only decode the newly generated tokens, not the input prompt
    input_length = inputs['input_ids'].shape[1]
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    print(f"Response: {response}\n")
    return response

# Test the model with a sample prompt
print("Testing SmolLM:")
print("="*60)
_ = test_model_generation("Hello! What is Machine Learning?")

## Mini-Tutorial: Key Concepts for Neural Archaeology

Before diving into the analysis, let's understand the key terms we'll be working with:

#### **What are Tokens?**
Think of tokens as the "words" a language model actually sees. Just like how you break a sentence into words, language models break text into **tokens**, which can be words, parts of words, or even single characters.

**Example:** The sentence "I think your code is overpythonized" might be split into tokens like:
- `["I", " think", " your", " code", " is", " over", "python", "ized"]`

*Learn more:* [OpenAI Tokenizer Tool](https://platform.openai.com/tokenizer) | [Article](https://quickchat.ai/post/tokens-entropy-question)

#### **What are Hidden States and Layers?**
Hidden states are the **internal thoughts** of the neural network at each layer. When the model processes a token, it creates a numerical representation (a vector) that captures what it "knows" about that token at that point in processing.

A language model is organized into **layers** stacked on top of each other - like floors in a building. Each layer refines and transforms the representation from the previous layer. Typically:

- **Early layers:** Capture basic patterns (spelling, grammar)
- **Middle layers:** Capture meaning and relationships
- **Late layers:** Capture abstract concepts and task-specific information

*Visualization:* [Jay Alammar's Hidden States Guide](http://jalammar.github.io/hidden-states/)

## Section 1: First Contact - Observing Natural Neural Patterns

Before we can manipulate a system, we must understand it. Let's observe how information flows through the model's layers and how different concepts create distinct neural signatures.

### Creating Diverse Test Data

We'll create a diverse dataset that includes safety-related content, emotional scenarios, and neutral text to explore how the model represents different types of information.

In [ ]:
# Create a much larger and more diverse test set
exploration_texts = []
exploration_labels = []

# Use circuit breaker data for clear safety contrast
print("Loading safety samples from circuit breaker data...")
n_safety = min(50, len(cb_data['train']['prompts']))

# Group UNSAFE together
unsafe_texts = []
for i in range(n_safety):
    unsafe_text = cb_data['train']['prompts'][i] + " " + cb_data['train']['unsafe_outputs'][i][:100]
    # Note: [:100] cutoff captures key semantic differences efficiently:
    # >90% of safe/unsafe outputs contain refusal/harmful content in first 100 chars
    unsafe_texts.append(unsafe_text)
    exploration_texts.append(unsafe_text)
    exploration_labels.append('unsafe')

# Group SAFE together
safe_texts = []
for i in range(n_safety):
    safe_text = cb_data['train']['prompts'][i] + " " + cb_data['train']['safe_outputs'][i][:100]
    safe_texts.append(safe_text)
    exploration_texts.append(safe_text)
    exploration_labels.append('safe')

# Emotional content - use more samples
# Pick only contrasting emotions (high arousal negative vs positive)
print("Loading emotion samples...")
emotions_to_use = ['joy','anger']
# emotions_to_use = []
for emotion in emotions_to_use:
    if emotion in emotion_data:
        # Use up to 15 samples per emotion
        n_emotion_samples = min(30, len(emotion_data[emotion]))
        for scenario in emotion_data[emotion][:n_emotion_samples]:
            exploration_texts.append(scenario)
            exploration_labels.append(f'emotion_{emotion}')




# Add more diverse neutral content
print("Adding neutral samples...")
neutral_samples = [
    "The capital of France is Paris.",
    "Water boils at 100 degrees Celsius.",
    "There are seven days in a week.",
    "Mathematics is the study of numbers and patterns.",
    "The Earth orbits around the Sun.",
    "Photosynthesis converts light into chemical energy.",
    "The Pacific Ocean is the largest ocean.",
    "Shakespeare wrote Romeo and Juliet.",
    "The human body has 206 bones.",
    "Gold is a chemical element with symbol Au.",
    "Mount Everest is the highest mountain.",
    "The speed of light is approximately 300,000 km/s.",
    "DNA contains genetic instructions.",
    "The Great Wall of China is visible from space is a myth.",
    "Chess is a strategic board game.",
    "Python is a programming language.",
    "The Amazon rainforest produces oxygen.",
    "Gravity causes objects to fall.",
    "The moon affects ocean tides.",
    "Atoms are made of protons, neutrons, and electrons."
]
for text in neutral_samples:
    exploration_texts.append(text)
    exploration_labels.append('neutral')

print(f"\nDataset composition:")
print(f"  Total samples: {len(exploration_texts)}")
print(f"  Categories: {len(set(exploration_labels))}")
for category in set(exploration_labels):
    count = exploration_labels.count(category)
    print(f"    {category}: {count} samples")

## **Question 1:** Exploring Pooling Strategies (15 points)

This question explores how different **pooling methods** affect a model's ability to distinguish between conceptual categories (e.g., safe vs. unsafe). You will analyze the hidden states from each layer of a language model to find the best approach.

### **Key Concepts**

* **Pooling**: The process of converting the hidden states for all tokens in an input into a single representative vector. You will test four methods:
    * **`last`**: Use the final token's hidden state.
    * **`mean`**: Average the hidden states of all tokens.
    * **`max`**: Take the element-wise maximum across all token hidden states.
    * **`first`**: Use the first token's hidden state (often a special `[CLS]` token).

* **Separation (Sep)**: A metric that measures how well the model's representations distinguish between categories. It's calculated as `Within-Category Similarity - Between-Category Similarity`.
    * A high separation score (e.g., > 0.3) indicates that representations for the same category are tightly clustered while representations for different categories are far apart.

### Tasks:
- Experiment with different pooling methods: 'last', 'mean', 'max', and 'first'.
- For each method, generate the neural landscape visualization and record the separation (Sep) values for each layer.
- Compare how pooling strategies affect category separation across layers.
- Identify the layer that provides the strongest category separation.
- Recommend the most effective pooling method for this task, explaining your reasoning.

### ANALYSIS:
-	Which pooling method yields the highest average separation (Sep) across layers?
-	Which layer demonstrates the most distinct clustering between categories?
      (Note: SmolLM has 24 layers numbered 0-23)
-	Based on your findings, which pooling strategy would you recommend and why?

In [ ]:
# ============================================================
# TODO: Question 1 - Pooling Strategy Experiment
# ============================================================
# Test each pooling method and record separation scores

pooling_methods = ['last', 'mean', 'max', 'first']
pooling_results = {}

for pooling_method in pooling_methods:
    print(f"\n{'='*60}")
    print(f"Testing pooling method: {pooling_method}")
    print(f"{'='*60}")
    
    # YOUR CODE HERE: Extract hidden states with this pooling method
    # Hint: Use get_hidden_states() with pooling=pooling_method
    # hidden_states = get_hidden_states(...)
    
    # YOUR CODE HERE: Visualize the neural landscape
    # Hint: Use visualize_neural_landscape()
    
    # YOUR CODE HERE: Record the separation scores for each layer
    # Store in pooling_results[pooling_method] = {...}
    pass

# YOUR ANALYSIS HERE (answer the 3 questions in markdown cell below)

### Required Analysis for Question 1

Based on your pooling strategy experiments, answer the following:

1. **Best Pooling Method (5 points)**: Which pooling method yields the highest average separation (Sep) across layers? Provide specific numbers.

2. **Layer Analysis (5 points)**: Which layer demonstrates the most distinct clustering between categories? (Note: SmolLM has 24 layers numbered 0-23). What does this tell you about where safety information is encoded?

3. **Recommendation (5 points)**: Based on your findings, which pooling strategy would you recommend for this task and why? Consider both performance and computational efficiency.

**YOUR ANALYSIS HERE:**
[Write your response here - minimum 4-5 sentences]

**The Neural Probe: Extracting Hidden States:** This function is our primary tool for examining the model's internal representations. It extracts the activation patterns at different layers of the network.

In [ ]:
def get_hidden_states(
    model,
    tokenizer,
    texts: List[str],
    layers: List[int] = None,
    token_pos: int = -1,
    batch_size: int = 8,
    max_length: int = 256,
    pooling: str = 'last',
    return_attention_mask: bool = False,
    enable_grad: bool = False,
    normalize_activations: bool = False  # ⭐ NEW: Add this parameter
) -> Union[Dict[int, np.ndarray], Tuple[Dict[int, np.ndarray], torch.Tensor]]:
    """
    Extract hidden states with multiple pooling strategies.

    NEW: normalize_activations - If True, normalize each hidden state to unit norm
         This makes separation scores comparable across architectures!
    """
    if layers is None:
        layers = list(range(model.config.num_hidden_layers))

    all_hidden_states = {layer: [] for layer in layers}
    all_attention_masks = [] if return_attention_mask else None

    grad_context = torch.enable_grad() if enable_grad else torch.no_grad()

    with grad_context:
        for i in tqdm(range(0, len(texts), batch_size), desc="Scanning neural activity"):
            batch_texts = texts[i:i+batch_size]

            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            ).to(model.device)

            outputs = model(**inputs, output_hidden_states=True)

            if return_attention_mask:
                all_attention_masks.append(inputs.attention_mask.cpu())

            for layer_idx in layers:
                hidden = outputs.hidden_states[layer_idx]

                # Different pooling strategies
                if pooling == 'last':
                    seq_lens = inputs.attention_mask.sum(dim=1) - 1
                    batch_indices = torch.arange(hidden.size(0), device=hidden.device)
                    selected_hidden = hidden[batch_indices, seq_lens]

                elif pooling == 'mean':
                    mask = inputs.attention_mask.unsqueeze(-1).expand(hidden.size()).float()
                    selected_hidden = (hidden * mask).sum(dim=1) / mask.sum(dim=1)

                elif pooling == 'max':
                    mask = inputs.attention_mask.unsqueeze(-1).expand(hidden.size())
                    hidden_masked = hidden.clone()
                    hidden_masked[~mask.bool()] = -float('inf')
                    selected_hidden = hidden_masked.max(dim=1)[0]

                elif pooling == 'first':
                    selected_hidden = hidden[:, 0]

                else:
                    selected_hidden = hidden[:, token_pos]

                # ⭐ NEW: Normalize activations if requested
                if normalize_activations:
                    norms = torch.norm(selected_hidden, dim=-1, keepdim=True)
                    selected_hidden = selected_hidden / (norms + 1e-8)

                # Convert to numpy unless gradients are needed
                if enable_grad:
                    all_hidden_states[layer_idx].append(selected_hidden)
                else:
                    all_hidden_states[layer_idx].append(selected_hidden.cpu().float().numpy())

            if i % (batch_size * 10) == 0:
                clear_memory()

    # Stack results
    for layer_idx in layers:
        if enable_grad:
            all_hidden_states[layer_idx] = torch.cat(all_hidden_states[layer_idx], dim=0)
        else:
            all_hidden_states[layer_idx] = np.vstack(all_hidden_states[layer_idx])

    if return_attention_mask:
        attention_masks = torch.cat(all_attention_masks, dim=0)
        return all_hidden_states, attention_masks

    return all_hidden_states

**Visualizing the Neural Landscape**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_neural_landscape(hidden_states, labels, layers_to_plot=None):
    """
    Create a visual map of neural activation patterns.

    This is like creating an fMRI scan of the model's "brain" -
    different concepts light up different regions. We're looking for
    the emergence of structure: do similar concepts cluster together?
    Do different layers organize information differently?
    """
    if layers_to_plot is None:
        # Default to plotting the first 6 layers available
        layers_to_plot = list(hidden_states.keys())[:6]

    fig = plt.figure(figsize=(20, 14))
    fig.suptitle("Neural Activation Landscape: How Concepts Organize Across Layers",
                 fontsize=16, fontweight='bold')

    # Group labels by category for better visualization
    label_groups = {}
    for i, label in enumerate(labels):
        category = label.split('_')[0] if '_' in label else label
        if category not in label_groups:
            label_groups[category] = []
        label_groups[category].append(i)

    # Reorder indices by category to group similar concepts together in the plot
    ordered_indices = []
    ordered_labels = []
    for category, indices in sorted(label_groups.items()):
        ordered_indices.extend(indices)
        ordered_labels.extend([labels[i] for i in indices])

    # Store similarity statistics for analysis
    similarity_stats = []

    for idx, layer in enumerate(layers_to_plot):
        plt.subplot(2, 3, idx + 1)

        hidden = hidden_states[layer]

        # Reorder by category for visual clarity
        hidden_ordered = hidden[ordered_indices]

        # --- Center the hidden states ---
        # Centering the hidden states by subtracting the mean vector is crucial.
        # It removes the common "background" activation, allowing the cosine
        # similarity to focus on the actual semantic differences between concepts.
        # This prevents an artificial increase in similarity scores.
        hidden_centered = hidden_ordered - hidden_ordered.mean(axis=0, keepdims=True)


        # Compute cosine similarity matrix using the centered states
        hidden_norm = hidden_centered / (np.linalg.norm(hidden_centered, axis=1, keepdims=True) + 1e-8)
        similarity = hidden_norm @ hidden_norm.T

        # Calculate within/between group similarities for better scaling and analysis
        within_sim = []
        between_sim = []
        # Create a mapping from original index to its group for easier lookup
        index_to_group = {idx: cat for cat, indices in label_groups.items() for idx in indices}

        for i in range(len(ordered_indices)):
            for j in range(i + 1, len(ordered_indices)):
                idx1 = ordered_indices[i]
                idx2 = ordered_indices[j]
                if index_to_group[idx1] == index_to_group[idx2]:
                    within_sim.append(similarity[i, j])
                else:
                    between_sim.append(similarity[i, j])

        # Calculate statistics
        within_mean = np.mean(within_sim) if within_sim else 0
        between_mean = np.mean(between_sim) if between_sim else 0
        separation = within_mean - between_mean

        # Enhanced color scaling based on actual data distribution
        # Use percentiles to create a robust color map against outliers
        sim_flat = similarity[np.triu_indices_from(similarity, k=1)]  # Upper triangle only
        vmin = np.percentile(sim_flat, 5)  # 5th percentile
        vmax = np.percentile(sim_flat, 95)  # 95th percentile

        # If the range is too small, expand it to ensure the visualization is meaningful
        if vmax - vmin < 0.1:
            mean_sim = np.mean(sim_flat)
            range_expand = max(0.2, (vmax - vmin) * 2)
            vmin = mean_sim - range_expand / 2
            vmax = mean_sim + range_expand / 2

        # Create clustered heatmap with enhanced scaling
        im = plt.imshow(similarity, cmap='RdBu_r', vmin=vmin, vmax=vmax, aspect='auto')
        plt.colorbar(im, fraction=0.046, pad=0.04)

        # Add category dividers to make the structure more visible
        current_pos = 0
        for category, indices in sorted(label_groups.items()):
            next_pos = current_pos + len(indices)
            if next_pos < len(ordered_labels):
                plt.axhline(y=next_pos - 0.5, color='green', linewidth=2, alpha=0.7)
                plt.axvline(x=next_pos - 0.5, color='green', linewidth=2, alpha=0.7)
            current_pos = next_pos

        # Title with summary statistics
        plt.title(f"Layer {layer}: {'Early' if layer < 8 else 'Middle' if layer < 16 else 'Late'}\n" +
                  f"Sep={separation:.3f} (W:{within_mean:.3f}, B:{between_mean:.3f})",
                  fontsize=10)

        # Add category labels on y-axis for readability
        category_positions = []
        category_names = []
        current_pos = 0
        for category, indices in sorted(label_groups.items()):
            mid_point = current_pos + len(indices) // 2
            category_positions.append(mid_point)
            category_names.append(category)
            current_pos += len(indices)

        plt.yticks(category_positions, category_names, fontsize=8)
        plt.xticks([])

        # Add text annotation to highlight key observations
        if idx == 0: # First plotted layer
            plt.text(0.02, 0.98, 'Token-level\nprocessing',
                     transform=plt.gca().transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round',
                                                        facecolor='wheat', alpha=0.5))
        elif idx == len(layers_to_plot) - 1: # Last plotted layer
            plt.text(0.02, 0.98, f'Semantic\nclusters\n(vmin:{vmin:.2f})',
                     transform=plt.gca().transAxes, fontsize=8,
                     verticalalignment='top', bbox=dict(boxstyle='round',
                                                        facecolor='lightgreen', alpha=0.5))

        similarity_stats.append({
            'layer': layer,
            'within_sim': within_mean,
            'between_sim': between_mean,
            'separation': separation,
            'vmin': vmin,
            'vmax': vmax,
            'range': vmax - vmin
        })

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
    plt.savefig('results/visualizations/neural_landscape_centered.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Print insights with statistics
    print("\nKEY OBSERVATIONS FROM NEURAL LANDSCAPE (WITH CENTERING):")
    print("-" * 70)
    print("Similarity Statistics by Layer:")
    for stat in similarity_stats:
        print(f"  Layer {stat['layer']:2d}: Separation={stat['separation']:+.4f} "
              f"(Within={stat['within_sim']:.4f}, Between={stat['between_sim']:.4f}) "
              f"Range=[{stat['vmin']:.3f}, {stat['vmax']:.3f}]")

    # Note about small sample size
    if len(labels) < 50:
        print(f"\nNote: Only {len(labels)} samples. Increase sample size for clearer patterns.")


**ANALYSIS WITH: SmolLM-1.7B-Instruct (24 layers)**

In [ ]:
# Extract hidden states with different pooling strategies
# Recalculate layer config for current model to avoid stale config
LAYER_CONFIG = get_layer_ranges(model)
exploration_layers = LAYER_CONFIG['exploration']
print(f"Exploring layers: {exploration_layers}")
print(f"  Early: {LAYER_CONFIG['early']}")
print(f"  Middle: {LAYER_CONFIG['middle']}")
print(f"  Late: {LAYER_CONFIG['late']}")

# Try different pooling methods
pooling_method = 'last'  # Change this to test: 'last', 'mean', 'max', 'first'
print(f"\nExtracting hidden states using '{pooling_method}' pooling...")
exploration_hidden = get_hidden_states(
    model, tokenizer, exploration_texts,
    layers=exploration_layers, batch_size=8,
    pooling=pooling_method
)
visualize_neural_landscape(exploration_hidden, exploration_labels)



### Discovering Emergent Structure: The Geometry of Thought


In [ ]:
def analyze_representation_geometry(hidden_states, labels):
    """
    Analyze the geometric structure of representations.

    In neuroscience, we've discovered that the brain organizes similar
    concepts in nearby regions. This function tests if language models
    exhibit the same property. We measure:
    - Within-category similarity (how close are examples of the same type?)
    - Between-category similarity (how separated are different types?)
    - Emergence of structure (does organization improve with depth?)
    """
    results = []

    for layer in hidden_states.keys():
        H = hidden_states[layer]

        # Normalize for angular distance
        H_norm = H / (np.linalg.norm(H, axis=1, keepdims=True) + 1e-8)

        # Compute category statistics
        unique_labels = list(set(labels))
        category_map = {label: np.array([l == label for l in labels]) for label in unique_labels}

        # Within-category similarity
        within_sims = []
        for label, mask in category_map.items():
            if mask.sum() > 1:
                category_hidden = H_norm[mask]
                sims = category_hidden @ category_hidden.T
                # Get upper triangle (excluding diagonal)
                upper_triangle = np.triu_indices(sims.shape[0], k=1)
                within_sims.extend(sims[upper_triangle])

        # Between-category similarity
        between_sims = []
        label_list = list(category_map.keys())
        for i, label1 in enumerate(label_list):
            for label2 in label_list[i+1:]:
                mask1 = category_map[label1]
                mask2 = category_map[label2]
                if mask1.sum() > 0 and mask2.sum() > 0:
                    sims = H_norm[mask1] @ H_norm[mask2].T
                    between_sims.extend(sims.flatten())

        within_mean = np.mean(within_sims) if within_sims else 0
        between_mean = np.mean(between_sims) if between_sims else 0
        separation = within_mean - between_mean

        # Compute cluster quality metric (silhouette coefficient approximation)
        cluster_quality = (within_mean - between_mean) / max(within_mean, between_mean) if max(within_mean, between_mean) > 0 else 0

        results.append({
            'layer': layer,
            'within_similarity': within_mean,
            'between_similarity': between_mean,
            'separation_score': separation,
            'cluster_quality': cluster_quality
        })

    return results

geometry_results = analyze_representation_geometry(exploration_hidden, exploration_labels)

# Visualize the emergence of structure with enhanced insights
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

layers = [r['layer'] for r in geometry_results]
within = [r['within_similarity'] for r in geometry_results]
between = [r['between_similarity'] for r in geometry_results]
separation = [r['separation_score'] for r in geometry_results]

# Plot 1: Similarity evolution
ax = axes[0]
ax.plot(layers, within, 'o-', label='Within Category', color='green', linewidth=2, markersize=8)
ax.plot(layers, between, 's-', label='Between Categories', color='red', linewidth=2, markersize=8)
ax.fill_between(layers, between, within, alpha=0.2, color='green')
ax.set_xlabel('Layer Depth', fontsize=12)
ax.set_ylabel('Average Cosine Similarity', fontsize=12)
ax.set_title('Emergence of Categorical Structure', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1])

# Plot 2: Separation score
ax = axes[1]
ax.plot(layers, separation, 'o-', color='purple', linewidth=2, markersize=8)
ax.set_xlabel('Layer Depth', fontsize=12)
ax.set_ylabel('Separation Score', fontsize=12)
ax.set_title('Concept Separation Across Layers', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax.fill_between(layers, 0, separation, alpha=0.3, color='purple')

# Mark regions of processing (24 layers total)
ax.axvspan(0, 5, alpha=0.1, color='blue')
ax.text(2.5, ax.get_ylim()[1]*0.9, 'Syntactic', ha='center', fontsize=10)
ax.axvspan(6, 15, alpha=0.1, color='green')
ax.text(10.5, ax.get_ylim()[1]*0.9, 'Semantic', ha='center', fontsize=10)
ax.axvspan(16, 23, alpha=0.1, color='red')
ax.text(19.5, ax.get_ylim()[1]*0.9, 'Abstract', ha='center', fontsize=10)

plt.suptitle('The Geometry of Thought: How Structure Emerges in Neural Networks',
            fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('results/visualizations/representation_geometry.png', dpi=150, bbox_inches='tight')
plt.show()

## **Question 2:** PCA Implementation & Validation (20 points)

### Understanding the Mathematics of Dimensionality Reduction

Let's implement PCA ourselves to discover structure in high-dimensional data.


### Your Tasks:
1. Complete the implementation of pca_from_scratch() and project_onto_components() functions.
Follow the provided step-by-step comments in the code template.
2. Run validate_pca_implementation() to test your PCA against several correctness checks (mean centering, orthogonality, normalization, etc.).
3. Ensure your PCA implementation passes at least 4 out of 5 tests to receive full credit.


### Required Analysis (5 points):

Answer the following questions briefly:

1. **Data Centering (2 points)**: Why do we center the data before computing PCA?

2. **Explained Variance (2 points)**: What does the explained variance ratio represent in PCA?

3. **Equal Eigenvalues (1 point)**: If two components have almost equal eigenvalues, what does that indicate about the data structure?

**YOUR ANALYSIS HERE:**
[Write your response here - 3-4 sentences minimum]

In [ ]:
def pca_from_scratch(H: np.ndarray, n_components: int = 2) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    TODO: Complete the PCA implementation.

    Principal Component Analysis finds the directions of maximum variance
    in your data. These directions often correspond to meaningful concepts.

    Steps to implement:
    1. Center the data (subtract mean)
    2. Compute covariance matrix
    3. Find eigenvalues and eigenvectors
    4. Sort by importance (largest eigenvalue first)
    5. Select top components
    6. Normalize eigenvectors to unit length

    Args:
        H: Data matrix (n_samples, n_features)
        n_components: Number of components to keep

    Returns:
        components: Principal directions (n_components, n_features)
        mean: Mean of original data
        explained_variance_ratio: Fraction of variance explained by each component
    """
    # Step 1: Center the data (remove mean)
    # TODO: Compute the mean along axis 0 and subtract it
       # YOUR CODE HERE
       # YOUR CODE HERE

    # Step 2: Compute covariance matrix
    # Cov = (1/(n-1)) * X^T @ X
    n_samples = H.shape[0]
       # YOUR CODE HERE

    # Step 3: Find eigenvalues and eigenvectors
    # The eigenvectors of the covariance matrix are the principal components
    # Hint: use np.linalg.eigh for symmetric matrices
       # YOUR CODE HERE

    # Step 4: Sort by importance (largest eigenvalue first)
    # TODO: Get indices that would sort eigenvalues in descending order
       # YOUR CODE HERE
    

    # Step 5: Select top components and ensure they're unit vectors
       # YOUR CODE HERE - select and transpose

    # Step 6: Calculate explained variance ratio
       # YOUR CODE HERE

    return components, mean, explained_variance_ratio

def project_onto_components(H: np.ndarray, components: np.ndarray, mean: np.ndarray) -> np.ndarray:
    """
    Project data onto discovered components.

    Args:
        H: Data to project
        components: Principal components (rows are components)
        mean: Mean to center data

    Returns:
        Projected data in component space
    """
    H_centered = H - mean
    return H_centered @ components.T

**Validation for PCA**:

Test your PCA implementation to ensure correctness.


In [ ]:
def validate_pca_implementation():
    """
    Unit tests for PCA implementation.
    This will verify your implementation is correct.
    """
    print("Running PCA validation tests...")
    tests_passed = []

    try:
        # Test 1: Known covariance matrix
        X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
        components, mean, var_ratio = pca_from_scratch(X, n_components=2)

        # Test mean centering
        assert np.allclose(mean, X.mean(axis=0)), "Mean calculation incorrect"
        tests_passed.append("mean_centering")
        print("  Test 1 passed: Mean centering")

        # Test 2: Components are orthogonal
        if len(components) > 1:
            dot_product = np.dot(components[0], components[1])
            assert np.abs(dot_product) < 1e-10, "Components not orthogonal"
            tests_passed.append("orthogonality")
            print("  Test 2 passed: Orthogonality")

        # Test 3: Components have unit norm
        for i, comp in enumerate(components):
            norm = np.linalg.norm(comp)
            assert np.allclose(norm, 1.0), f"Component {i} not unit norm (norm={norm})"
        tests_passed.append("unit_norm")
        print("  Test 3 passed: Unit norm")

        # Test 4: Variance ratio sums to <= 1
        assert var_ratio.sum() <= 1.01, "Variance ratio exceeds 1"
        tests_passed.append("variance_sum")
        print("  Test 4 passed: Variance sum")

        # Test 5: Eigenvalues are sorted in descending order
        I = np.eye(5)
        comp_I, mean_I, var_I = pca_from_scratch(I, n_components=3)
        assert all(var_I[i] >= var_I[i+1] for i in range(len(var_I)-1)), "Eigenvalues not sorted correctly"
        tests_passed.append("sorting")
        print("  Test 5 passed: Sorting")

    except Exception as e:
        print(f"  Test failed with error: {e}")

    score = len(tests_passed) / 5
    print(f"\nPCA Implementation Score: {len(tests_passed)}/5 tests passed ({score*100:.0f}%)")
    print(f"Tests passed: {tests_passed}")

    # Assertion for grading
    assert score >= 0.8, "PCA implementation needs more work (less than 80% tests passed)"

    return score

# Run validation - this must pass for full credit
pca_validation_score = validate_pca_implementation()

## **Question 3:** K-Means Clustering Analysis (20 points)

### Discovering Natural Groupings with K-Means

Compare how well K-means clustering can discover emotion categories with and without dimensionality reduction.


In [ ]:
def kmeans_from_scratch(X: np.ndarray, n_clusters: int, max_iters: int = 100, random_state: int = 42) -> Tuple[np.ndarray, np.ndarray]:
    """
    TODO: Implement the K-Means clustering algorithm from scratch.

    Steps:
    1. Initialize centroids: Randomly select 'n_clusters' points from X as initial centroids.
       (Hint: Use np.random.RandomState(random_state).permutation)
    2. Loop for 'max_iters':
       a. Assign clusters: For each point, find the nearest centroid (use Euclidean distance).
          (Hint: np.linalg.norm(point - centroid, axis=1))
       b. Update centroids: Compute the mean of all points assigned to each cluster.
    3. Return the final centroids and the cluster assignments (labels) for each point.

    Args:
        X: Data matrix (n_samples, n_features)
        n_clusters: The number of clusters (K)
        max_iters: Maximum number of iterations
        random_state: Seed for reproducibility

    Returns:
        centroids: Final cluster centroids (n_clusters, n_features)
        labels: Cluster assignment for each point (n_samples,)
    """
    np.random.seed(random_state)
    n_samples, n_features = X.shape

    # --- YOUR CODE HERE ---

    # Step 1: Initialize centroids
    # Hint: Randomly select indices
     

    labels = np.zeros(n_samples)

    for _ in range(max_iters):
        # Step 2a: Assign clusters
        for i in range(n_samples):
            # Calculate distances from point i to all centroids
            distances =  
            # Assign to closest centroid
            labels[i] =  

        # Step 2b: Update centroids
        new_centroids = np.zeros((n_clusters, n_features))
        for k in range(n_clusters):
            cluster_points =  
            if len(cluster_points) > 0:
                new_centroids[k] =  
            else:
                # Re-initialize empty clusters
                new_centroids[k] = 

        # Check for convergence (if centroids don't change)
        if np.allclose(centroids, new_centroids):
            break

        centroids = new_centroids

    # --- END YOUR CODE ---

    return centroids, labels

In [ ]:
def kmeans_emotion_analysis(emotion_hidden, emotion_labels, layer=20):
    """
    TODO: Compare clustering approaches for emotion discovery.

    Tasks:
    1. Apply K-means directly to raw hidden states
    2. Apply PCA first, then K-means on reduced features
    3. Compare clustering quality using Adjusted Rand Index (ARI)
    4. Analyze which emotions cluster well together

    Expected outputs:
    - ARI scores for both approaches
    - Silhouette scores for cluster quality
    - Confusion matrix showing predicted vs true labels

    Evaluation Metrics:
    - adjusted_rand_score: Measures agreement between true and predicted labels
      (see: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html)
    - silhouette_score: Measures cluster cohesion and separation
      (see: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

    Returns:
        Dictionary with clustering metrics and analysis
    """
    from sklearn.cluster import KMeans # Note: You can use this for comparison, but you are requried to implement KMeans yourself!!
    from sklearn.metrics import adjusted_rand_score, silhouette_score
    # Note: While you are not required to implement these scores
    # Visit the documentation links above to understand how these metrics are calculated and what they measure.

    results = {}

    # Get hidden states for specified layer
    H_emotions = emotion_hidden[layer]

    # Map emotion labels to integers for evaluation
    unique_emotions = list(set(emotion_labels))
    emotion_to_id = {e: i for i, e in enumerate(unique_emotions)}
    true_labels = np.array([emotion_to_id[e] for e in emotion_labels])
    n_emotions = len(unique_emotions)

    # TODO: Task 1 - Direct K-means on raw features
    # Apply K-means with n_clusters=n_emotions
    #YOUR Implementation
    centroids_raw, clusters_raw =  

    #REFERENCE FOR COMPARISON
    kmeans_raw_sklearn =    # YOUR CODE HERE
    clusters_raw_sklearn =    # YOUR CODE HERE

    # TODO: Task 2 - PCA + K-means
    # First reduce dimensions with PCA (try n_components=10)
    # Then apply K-means
    n_pca_components = min(10, H_emotions.shape[0] - 1)
    components, mean, var_ratio =    # YOUR CODE HERE - use your pca_from_scratch
    H_pca =   # YOUR CODE HERE - project data

    #YOUR KMEANS IMPLEMENTATION
    centroids_pca, clusters_pca = kmeans_from_scratch(H_pca, n_clusters=n_emotions, random_state=42)
    #REFENRENCE FOR COMPARISON
    kmeans_pca_sklearn =    # YOUR CODE HERE
    clusters_pca_sklearn =    # YOUR CODE HERE

    # TODO: Task 3 - Compute metrics
    # Use adjusted_rand_score and silhouette_score
    # --- Metrics for YOUR implementation (for grading) ---
    results['raw_ari'] =   # YOUR CODE HERE
    results['pca_ari'] =   # YOUR CODE HERE
    results['raw_silhouette'] =    # YOUR CODE HERE
    results['pca_silhouette'] =    # YOUR CODE HERE
    # --- Metrics for sklearn reference (for your comparison) ---
    results['raw_ari_sklearn'] = adjusted_rand_score(true_labels, clusters_raw_sklearn)
    results['pca_ari_sklearn'] = adjusted_rand_score(true_labels, clusters_pca_sklearn)
    results['raw_silhouette_sklearn'] = silhouette_score(H_emotions, clusters_raw_sklearn)
    results['pca_silhouette_sklearn'] = silhouette_score(H_pca, clusters_pca_sklearn)

    # Determine winner
    results['best_method'] = 'raw' if results['raw_ari'] > results['pca_ari'] else 'pca'

    # Validation assertions
    assert results['raw_ari'] > -0.1, "ARI too low - check K-means implementation"
    assert results['raw_silhouette'] > 0, "Silhouette score invalid"

    return results



In [ ]:
# Prepare emotion data for clustering
emotion_texts = []
emotion_labels_clustering = []

for emotion in ['joy', 'sadness', 'anger', 'fear']:
    if emotion in emotion_data:
        for scenario in emotion_data[emotion][:200]:  # 100 samples per emotion
            emotion_texts.append(scenario)
            emotion_labels_clustering.append(emotion)

print(f"Prepared {len(emotion_texts)} emotional examples for clustering")

# Extract hidden states
# Use late layers for abstract emotion concepts
# Recalculate layer config for current model
LAYER_CONFIG = get_layer_ranges(model)
emotion_clustering_layers = LAYER_CONFIG['late'][-3:]  # Last 3 late layers
print(f"Using late layers for emotion clustering: {emotion_clustering_layers}")
emotion_hidden_clustering = get_hidden_states(
    model, tokenizer, emotion_texts,
    layers=emotion_clustering_layers, batch_size=16
)

# Run K-means analysis
# Use the middle layer from the extracted layers
analysis_layer = emotion_clustering_layers[len(emotion_clustering_layers)//2]
kmeans_results = kmeans_emotion_analysis(
    emotion_hidden_clustering,
    emotion_labels_clustering,
    layer=analysis_layer  # Use one of the extracted layers
)

# Display results
print("\nK-Means Clustering Results:")
print(f"Raw features ARI: {kmeans_results['raw_ari']:.3f}")
print(f"PCA features ARI: {kmeans_results['pca_ari']:.3f}")
print(f"Raw silhouette: {kmeans_results['raw_silhouette']:.3f}")
print(f"PCA silhouette: {kmeans_results['pca_silhouette']:.3f}")
print(f"Best method: {kmeans_results['best_method']}")

print(f"\n--- SKLEARN REFERENCE (Layer {analysis_layer}) ---")
print(f"  Raw features ARI: {kmeans_results['raw_ari_sklearn']:.3f}")
print(f"  PCA features ARI: {kmeans_results['pca_ari_sklearn']:.3f}")
print(f"  Raw silhouette: {kmeans_results['raw_silhouette_sklearn']:.3f}")
print(f"  PCA silhouette: {kmeans_results['pca_silhouette_sklearn']:.3f}")

### Required Analysis for Question 3 (5 points)

Based on your K-means clustering results, answer the following:

1. **Performance Comparison (2 points)**: Compare the performance of PCA-based vs raw clustering. What factors might explain the difference in performance? Consider both the nature of the data and the properties of each approach.

2. **Confusion Patterns (2 points)**: Examine which emotion pairs are frequently confused. What patterns do you notice? How might these confusions relate to psychological theories of emotion (e.g., valence-arousal models)?

3. **Silhouette Interpretation (1 point)**: Interpret the silhouette scores. What do they reveal about the natural clustering structure of emotions in the model's representation space?

**YOUR ANALYSIS HERE:**
[Write your response here - minimum 5-6 sentences]

## Section 2: Unsupervised Discovery - Finding Natural Cognitive Dimensions

Now we'll use the PCA implementation to discover how the model naturally organizes information.

### Applying PCA to Safety Representations

In [ ]:
# Prepare safety dataset
print("Discovering natural safety dimensions...")

safety_texts = []
safety_labels = []

# Mix safe and unsafe examples from circuit breaker data
n_samples = min(500, len(cb_data['train']['prompts']))
for i in range(n_samples):
    # Unsafe: prompt + unsafe response
    unsafe_text = cb_data['train']['prompts'][i] + " " + cb_data['train']['unsafe_outputs'][i][:50]
    safety_texts.append(unsafe_text)
    safety_labels.append('unsafe')

    # Safe: prompt + safe response
    safe_text = cb_data['train']['prompts'][i] + " " + cb_data['train']['safe_outputs'][i][:50]
    safety_texts.append(safe_text)
    safety_labels.append('safe')

print(f"Prepared {len(safety_texts)} safety examples")

# Extract hidden states
# Use exploration layers to match Q6 supervised analysis
# Recalculate layer config for current model
LAYER_CONFIG = get_layer_ranges(model)
safety_pca_layers = LAYER_CONFIG['exploration']  # Use same layers as Q6 for fair comparison
print(f"Using exploration layers for safety PCA: {safety_pca_layers}")
safety_hidden = get_hidden_states(model, tokenizer, safety_texts, layers=safety_pca_layers, batch_size=16)

# Apply PCA
safety_pca_results = {}
for layer in safety_pca_layers:
    H = safety_hidden[layer]
    components, mean, variance_ratio = pca_from_scratch(H, n_components=10)

    safety_pca_results[layer] = {
        'components': components,
        'mean': mean,
        'variance_ratio': variance_ratio,
        'projections': project_onto_components(H, components[:2], mean)
    }

    print(f"Layer {layer}: Top 2 PCs explain {variance_ratio[:2].sum():.1%} of variance")

### Applying PCA to Emotion Representations


In [ ]:
# Apply PCA to emotion data
print("\nDiscovering natural emotional dimensions...")

emotion_pca_results = {}
for layer in emotion_clustering_layers:
    H = emotion_hidden_clustering[layer]
    components, mean, variance_ratio = pca_from_scratch(H, n_components=10)

    emotion_pca_results[layer] = {
        'components': components,
        'mean': mean,
        'variance_ratio': variance_ratio,
        'projections': project_onto_components(H, components[:2], mean)
    }

    print(f"Layer {layer}: Top 2 PCs explain {variance_ratio[:2].sum():.1%} of variance")

## **Question 4:** Discovering Natural Emotional Dimensions (15 points)

**This is an ANALYSIS-ONLY question** - no new code implementation required.

Using the PCA visualizations already generated above for emotions, analyze and answer:

### Required Analysis (15 points):

1. **Separation Quality (4 points)**: How well do the discovered principal components separate different classes? Compare safe/unsafe separation to emotion category separation.

2. **Variance vs Usefulness (4 points)**: Emotions show higher variance explained in PC1 compared to safety. Does higher variance mean the components are more useful for distinguishing categories? Why or why not?

3. **What is PCA Finding? (4 points)**: If the emotion PCs don't cleanly separate emotion categories, what might they be capturing instead? Consider other sources of variation in the data (e.g., sentence length, topic, writing style, sentiment intensity).

4. **Comparison to Q3 (3 points)**: Does this explain why K-means struggled in Q3? When might unsupervised discovery work well vs poorly?

**Write your analysis below (minimum 5-7 sentences total):**

**YOUR ANALYSIS HERE:**
[Write your response here]

### Enhanced PCA Visualization

In [ ]:
def visualize_pca_discoveries(safety_results, emotion_results, layer=None):
    """
    Visualize PCA discoveries. If layer is None, uses the last layer from LAYER_CONFIG.
    """
    if layer is None:
        layer = LAYER_CONFIG['last_layer']
    """
    Visualize the natural dimensions discovered by PCA.

    Essential plots:
    - 2D scatter plots showing PC1 vs PC2 for safety and emotions
    - Variance explained to understand dimensionality
    - PC1 distribution to quantify separation quality
    """
    fig = plt.figure(figsize=(15, 5))

    # Safety PCA visualization
    ax = plt.subplot(1, 4, 1)
    projections = safety_results[layer]['projections']
    colors = ['red' if l == 'unsafe' else 'green' for l in safety_labels]
    scatter = ax.scatter(projections[:, 0], projections[:, 1], c=colors, alpha=0.5, s=20)
    ax.set_xlabel(f'PC1 ({safety_results[layer]["variance_ratio"][0]:.1%} var)')
    ax.set_ylabel(f'PC2 ({safety_results[layer]["variance_ratio"][1]:.1%} var)')
    ax.set_title('Safety: Natural Dimensions')
    ax.grid(True, alpha=0.3)

    # Add decision boundary
    from sklearn.svm import SVC
    svm = SVC(kernel='linear')
    y = [1 if l == 'safe' else 0 for l in safety_labels]
    svm.fit(projections[:, :2], y)

    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50),
                         np.linspace(ylim[0], ylim[1], 50))
    Z = svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contour(xx, yy, Z, colors='k', levels=[0], alpha=0.5, linestyles=['--'])

    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='red', alpha=0.5, label='Unsafe'),
                      Patch(facecolor='green', alpha=0.5, label='Safe')]
    ax.legend(handles=legend_elements, loc='best')

    # Emotion PCA visualization
    ax = plt.subplot(1, 4, 2)
    projections = emotion_results[layer]['projections']
    emotion_colors = {'joy': 'yellow', 'sadness': 'blue', 'anger': 'red', 'fear': 'purple'}
    colors = [emotion_colors.get(l, 'gray') for l in emotion_labels_clustering]
    ax.scatter(projections[:, 0], projections[:, 1], c=colors, alpha=0.5, s=20)
    ax.set_xlabel(f'PC1 ({emotion_results[layer]["variance_ratio"][0]:.1%} var)')
    ax.set_ylabel(f'PC2 ({emotion_results[layer]["variance_ratio"][1]:.1%} var)')
    ax.set_title('Emotions: Natural Dimensions')
    ax.grid(True, alpha=0.3)

    # Add emotion centroids with labels
    for emotion in emotion_colors.keys():
        mask = [l == emotion for l in emotion_labels_clustering]
        if any(mask):
            emotion_proj = projections[mask]
            centroid = emotion_proj.mean(axis=0)
            ax.scatter(centroid[0], centroid[1], s=200, c=emotion_colors[emotion],
                      edgecolors='black', linewidth=2, marker='*')
            ax.annotate(emotion, (centroid[0], centroid[1]), fontsize=9,
                       ha='center', va='bottom')

    # Cumulative variance
    ax = plt.subplot(1, 4, 3)
    x = range(1, 11)
    safety_cumvar = np.cumsum(safety_results[layer]['variance_ratio'])
    emotion_cumvar = np.cumsum(emotion_results[layer]['variance_ratio'])
    ax.plot(x, safety_cumvar, 'o-', color='blue', label='Safety', linewidth=2)
    ax.plot(x, emotion_cumvar, 's-', color='orange', label='Emotion', linewidth=2)
    ax.axhline(y=0.9, color='red', linestyle='--', alpha=0.5, label='90% threshold')
    ax.set_xlabel('Number of Components')
    ax.set_ylabel('Cumulative Variance Explained')
    ax.set_title('Dimensional Requirements')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # PC1 distribution analysis for safety
    ax = plt.subplot(1, 4, 4)
    pc1_scores = safety_results[layer]['projections'][:, 0]
    safe_scores = [pc1_scores[i] for i, l in enumerate(safety_labels) if l == 'safe']
    unsafe_scores = [pc1_scores[i] for i, l in enumerate(safety_labels) if l == 'unsafe']

    bins = np.linspace(min(pc1_scores), max(pc1_scores), 30)
    ax.hist(safe_scores, bins=bins, alpha=0.5, label='Safe', color='green', density=True)
    ax.hist(unsafe_scores, bins=bins, alpha=0.5, label='Unsafe', color='red', density=True)
    ax.set_xlabel('PC1 Score')
    ax.set_ylabel('Density')
    ax.set_title('PC1: The Safety Dimension')
    ax.legend()
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.3)

    # Calculate separation
    safe_mean = np.mean(safe_scores)
    unsafe_mean = np.mean(unsafe_scores)
    separation = abs(safe_mean - unsafe_mean) / (np.std(safe_scores) + np.std(unsafe_scores))
    ax.text(0.02, 0.95, f'Separation: {separation:.2f}σ',
           transform=ax.transAxes, fontsize=10,
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    plt.suptitle(f'Layer {layer}: Unsupervised Discovery of Cognitive Dimensions',
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'results/visualizations/pca_discoveries_layer_{layer}.png', dpi=150, bbox_inches='tight')
    plt.show()

# Visualize PCA results
# Note: Available layers are stored in the dictionaries. Check with:
# print(f"Available layers: {list(safety_pca_results.keys())}")
#
# EXPERIMENT: Try different layers to see how PCA patterns change across depth!
# - Early layers: More syntax/surface features
# - Late layers: More semantic/abstract concepts
#
# Option 1: Use None to auto-select the best layer
# Option 2: Choose a specific layer from the available ones
layer_to_visualize = list(safety_pca_results.keys())[-1]  # Use the last (deepest) layer
visualize_pca_discoveries(safety_pca_results, emotion_pca_results, layer=layer_to_visualize)

## Section 3: Supervised Learning - Targeted Neural Engineering

In Section 2, we used PCA to discover natural patterns in neural representations without any labels. While powerful, PCA doesn't know which patterns are relevant for our specific task (e.g., safety detection).

Now we shift to *supervised learning*, where we use labeled examples to engineer detectors optimized for specific concepts. This section explores two approaches of increasing sophistication:
1. **Mean Difference** (Simple Supervised): A baseline that subtracts class means
2. **Logistic Regression** (Optimized Supervised): Finds the optimal linear separator

Let's start with the simplest supervised approach to build intuition.

### Mean Difference Method: A Simple Baseline

**Key Idea:** If we have labeled examples of "safe" and "unsafe" text, the simplest
way to find a separating direction is:
1. Compute the average (mean) hidden state for all safe examples
2. Compute the average (mean) hidden state for all unsafe examples  
3. Subtract them: `direction = mean_safe - mean_unsafe`

This gives us a vector pointing from "unsafe" toward "safe" in the representation space.

**Limitation:** This method only uses the class means and ignores the variance and
distribution of the data. Logistic regression (which you'll implement in Q6) is more
sophisticated because it finds the *optimal* linear separator through optimization.

In [ ]:
def compute_mean_difference(H_pos: np.ndarray, H_neg: np.ndarray, normalize: bool = True) -> np.ndarray:
    """
    Compute the direction that separates two classes using mean difference.

    This is the simplest supervised method: just subtract the class means.
    It's like finding the neural signature that distinguishes between
    two mental states (e.g., safe vs unsafe).

    Args:
        H_pos: Hidden states for positive class (e.g., safe examples) [n_samples, hidden_dim]
        H_neg: Hidden states for negative class (e.g., unsafe examples) [n_samples, hidden_dim]
        normalize: Whether to normalize the direction to unit length

    Returns:
        direction: Vector pointing from negative to positive class [hidden_dim]
    """
    mean_pos = H_pos.mean(axis=0)  # Average over all positive examples
    mean_neg = H_neg.mean(axis=0)  # Average over all negative examples

    direction = mean_pos - mean_neg  # Direction from negative to positive

    if normalize:
        direction = direction / (np.linalg.norm(direction) + 1e-8)  # Normalize to unit length

    return direction

def project_onto_direction(H: np.ndarray, direction: np.ndarray) -> np.ndarray:
    """
    Project hidden states onto a direction vector.

    This gives us a score for each example: how much it aligns with the direction.
    Higher scores mean the example is more similar to the positive class.

    Args:
        H: Hidden states [n_samples, hidden_dim]
        direction: Direction vector [hidden_dim]

    Returns:
        scores: Projection scores [n_samples]
    """
    direction = direction / (np.linalg.norm(direction) + 1e-8)  # Ensure normalized
    return H @ direction  # Matrix multiplication gives dot product for each sample

## **Question 5:** Supervised Safety Detection (15 points)

### From Simple to Optimized: Logistic Regression

Now you'll implement **logistic regression**, which goes beyond simple averaging to find the *optimized* linear separator between classes.

**What you'll implement/analyze:**
1. **Data preparation**: Split labeled data into train/test sets
2. **Model training**: Train a logistic regression classifier
3. **Evaluation**: Compute accuracy, AUC, and other metrics
4. **Layer analysis**: Compare performance across different layers
5. **Method comparison**: See how much better optimization is vs. simple mean difference

In [ ]:
def train_safety_classifier(safety_texts, safety_labels, model, tokenizer, layer, test_size=0.2):
    """
    Train a supervised safety classifier using logistic regression.

    This function demonstrates the core supervised learning pipeline:
    extracting features, splitting data, training, and evaluation.

    Args:
        safety_texts: List[str] - Text examples (both safe and unsafe)
        safety_labels: List[str] - Labels ('safe' or 'unsafe') corresponding to texts
        model: The language model to extract hidden states from
        tokenizer: The tokenizer for the model
        layer: int - Which layer to extract hidden states from
        test_size: float - Fraction of data to use for testing (default: 0.2)

    Returns:
        results: dict containing:
            - 'classifier': Trained LogisticRegression model
            - 'accuracy': float - Test set accuracy
            - 'auc': float - Area under ROC curve
            - 'direction': np.ndarray - Learned weight vector (normalized)
            - 'X_test': Test features (for later visualization)
            - 'y_test': Test labels (for later visualization)
            - 'y_pred': Predictions on test set
            - 'y_prob': Prediction probabilities on test set
    """
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, roc_auc_score

    # ============================================================
    # STEP 1: Extract hidden states (features)
    # ============================================================
    print(f"Extracting hidden states from layer {layer}...")

    # TODO: Extract hidden states for all texts at the specified layer
    # Hint: Use get_hidden_states() function with layers=[layer]
    # Expected output: Dictionary with key=layer, value=numpy array of shape [n_samples, hidden_dim]

    # YOUR CODE HERE (1-3 lines)
    hidden_dict =  
    X =    # Shape: [n_samples, hidden_dim]

    print(f"  Extracted features shape: {X.shape}")

    # ============================================================
    # STEP 2: Prepare labels
    # ============================================================
    # Convert string labels to binary: 'safe' -> 1, 'unsafe' -> 0

    # TODO: Create binary labels array
    # Hint: Use list comprehension or np.array with conditional
    # Expected output: numpy array of shape [n_samples] with values 0 or 1

    # YOUR CODE HERE (1-2 lines)
    y =  

    print(f"  Labels shape: {y.shape}")
    print(f"  Class distribution: {np.sum(y == 0)} unsafe, {np.sum(y == 1)} safe")

    # ============================================================
    # STEP 3: Split into train and test sets
    # ============================================================
    # Use train_test_split to create training and testing sets

    # TODO: Split the data into train/test sets
    # Hint: Use train_test_split(X, y, test_size=test_size, random_state=42)
    # Expected output: X_train, X_test, y_train, y_test

    # YOUR CODE HERE (1 line)
    X_train, X_test, y_train, y_test =  

    print(f"  Train set: {X_train.shape[0]} samples")
    print(f"  Test set: {X_test.shape[0]} samples")

    # ============================================================
    # STEP 4: Train logistic regression classifier
    # ============================================================
    print("Training logistic regression classifier...")

    # TODO: Create and train a LogisticRegression classifier
    # Hint: Set max_iter to e.g., 1000
    # Then use .fit()
    # Expected output: Trained classifier object

    # YOUR CODE HERE (2 lines)
     

    # ============================================================
    # STEP 5: Make predictions and evaluate
    # ============================================================
    print("Evaluating on test set...")

    # TODO: Generate predictions and probabilities
    # Hint: Use classifier.predict(X_test) and classifier.predict_proba(X_test)
    # Expected output: y_pred (binary predictions), y_prob (probabilities for class 1)

    # YOUR CODE HERE (2 lines)
    y_pred =  
    y_prob =    # Probability of class 1 (safe)

    # TODO: Compute accuracy and AUC
    # Hint: Use accuracy_score(y_test, y_pred) and roc_auc_score(y_test, y_prob)
    # Expected output: accuracy (float), auc (float)

    # YOUR CODE HERE (2 lines)
    accuracy =  
    auc =  

    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  AUC: {auc:.3f}")

    # ============================================================
    # STEP 6: Extract learned direction vector
    # ============================================================
    # The logistic regression weights represent the "safety direction"
    # in the hidden state space

    # TODO: Extract and normalize the weight vector
    # Hint: classifier.coef_ gives the weights, use .flatten() to get 1D array
    # Then normalize: direction / np.linalg.norm(direction)
    # Expected output: direction (numpy array of shape [hidden_dim])

    # YOUR CODE HERE (2-3 lines)
    direction =  
    direction =    # Normalize to unit length

    # ============================================================
    # VALIDATION CHECKS
    # ============================================================
    assert X_train.shape[0] + X_test.shape[0] == len(safety_texts), "Data split error"
    assert y_pred.shape[0] == y_test.shape[0], "Prediction shape mismatch"
    assert 0 <= accuracy <= 1, "Accuracy out of range"
    assert 0 <= auc <= 1, "AUC out of range"
    assert direction.shape[0] == X.shape[1], "Direction dimension mismatch"
    assert abs(np.linalg.norm(direction) - 1.0) < 0.01, "Direction not normalized"

    # Package results
    results = {
        'classifier': classifier,
        'accuracy': accuracy,
        'auc': auc,
        'direction': direction,
        'X_test': X_test,
        'y_test': y_test,
        'y_pred': y_pred,
        'y_prob': y_prob
    }

    return results

In [ ]:
# ============================================================
# TEST YOUR IMPLEMENTATION
# ============================================================
print("="*70)
print("QUESTION 6: Testing Supervised Safety Classifier")
print("="*70)

# Recalculate layer config for current model to ensure compatibility
LAYER_CONFIG = get_layer_ranges(model, print_config=False)

# Select a layer to test (use a late layer where safety concepts emerge)
test_layer = LAYER_CONFIG['safety_layers'][-1]
print(f"\nTesting on layer {test_layer}...")

# Train the classifier
q6_results = train_safety_classifier(
    safety_texts=safety_texts,
    safety_labels=safety_labels,
    model=model,
    tokenizer=tokenizer,
    layer=test_layer,
    test_size=0.2
)

print(f"\n✓ Training complete!")
print(f"  Final accuracy: {q6_results['accuracy']:.3f}")
print(f"  Final AUC: {q6_results['auc']:.3f}")
print(f"  Direction vector shape: {q6_results['direction'].shape}")

In [ ]:
# ============================================================
# LAYER ANALYSIS (OPEN-ENDED EXPLORATION)
# ============================================================
print("\n" + "="*70)
print("LAYER ANALYSIS: Comparing Performance Across Layers")
print("="*70)

# TODO: Analyze how classification performance varies across layers
# Try training classifiers on different layers and compare results
# Suggested layers to try: early, middle, and late layers

# YOUR CODE HERE - Implement layer comparison
# Hint: Loop through different layers, call train_safety_classifier for each,
# and store the results. Then visualize accuracy/AUC across layers.

layer_comparison_results = {}

# Ensure layer config is up to date for current model
LAYER_CONFIG = get_layer_ranges(model, print_config=False)

# Example structure (you can modify):
layers_to_test = LAYER_CONFIG['exploration']  # or choose specific layers
print(f"\nTesting layers: {layers_to_test}")

for layer in layers_to_test:
    print(f"\n--- Layer {layer} ---")
    results = train_safety_classifier(
        safety_texts=safety_texts,
        safety_labels=safety_labels,
        model=model,
        tokenizer=tokenizer,
        layer=layer,
        test_size=0.2
    )
    layer_comparison_results[layer] = results

# Visualize results
layers = list(layer_comparison_results.keys())
accuracies = [layer_comparison_results[l]['accuracy'] for l in layers]
aucs = [layer_comparison_results[l]['auc'] for l in layers]

plt.figure(figsize=(14, 5))

# Plot 1: Accuracy across layers
plt.subplot(1, 2, 1)
plt.plot(layers, accuracies, 'o-', linewidth=2, markersize=8, color='steelblue')
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random baseline')
plt.xlabel('Layer Index', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.title('Safety Classification Accuracy by Layer', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

# Plot 2: AUC across layers
plt.subplot(1, 2, 2)
plt.plot(layers, aucs, 'o-', linewidth=2, markersize=8, color='darkorange')
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random baseline')
plt.xlabel('Layer Index', fontsize=12)
plt.ylabel('AUC Score', fontsize=12)
plt.title('Safety Classification AUC by Layer', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.savefig('results/student_analysis/q6_layer_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

# Find best layer
best_layer_acc = max(layer_comparison_results.keys(), key=lambda l: layer_comparison_results[l]['accuracy'])
best_layer_auc = max(layer_comparison_results.keys(), key=lambda l: layer_comparison_results[l]['auc'])

print("\n" + "="*70)
print("LAYER ANALYSIS SUMMARY")
print("="*70)
print(f"Best layer by accuracy: {best_layer_acc} (Acc={layer_comparison_results[best_layer_acc]['accuracy']:.3f})")
print(f"Best layer by AUC: {best_layer_auc} (AUC={layer_comparison_results[best_layer_auc]['auc']:.3f})")
print(f"\nLayer range tested: {min(layers)} to {max(layers)}")
print(f"Accuracy range: {min(accuracies):.3f} to {max(accuracies):.3f}")
print(f"AUC range: {min(aucs):.3f} to {max(aucs):.3f}")

In [ ]:
# ============================================================
# SUPERVISED VS UNSUPERVISED COMPARISON
# ============================================================
print("\n" + "="*70)
print("COMPARING SUPERVISED VS UNSUPERVISED APPROACHES")
print("="*70)

# Use the best performing layer from your analysis
best_layer = best_layer_auc
supervised_direction = layer_comparison_results[best_layer]['direction']
X_test = layer_comparison_results[best_layer]['X_test']
y_test = layer_comparison_results[best_layer]['y_test']
y_prob = layer_comparison_results[best_layer]['y_prob']

# Project test data onto supervised direction
supervised_projections = X_test @ supervised_direction

# Initialize PCA metrics
pca_available = best_layer in safety_pca_results
pca_auc = None
pca_projections = None

if pca_available:
    # Get PCA direction and compute projections
    pca_direction = safety_pca_results[best_layer]['components'][0]
    pca_mean = safety_pca_results[best_layer]['mean']
    pca_projections = (X_test - pca_mean) @ pca_direction

    # Flip sign if needed so safe is positive
    if np.mean(pca_projections[y_test == 1]) < np.mean(pca_projections[y_test == 0]):
        pca_projections = -pca_projections

    # Compute PCA AUC
    from sklearn.metrics import roc_auc_score, roc_curve
    pca_auc = roc_auc_score(y_test, pca_projections)

    # Compute cosine similarity between directions
    cosine_sim = np.dot(supervised_direction, pca_direction)

    print(f"\nDirection Comparison (Layer {best_layer}):")
    print(f"  Cosine similarity: {cosine_sim:.3f} ({abs(cosine_sim):.1%} alignment)")

    if abs(cosine_sim) > 0.7:
        print("  → Highly aligned (both found similar patterns)")
    elif abs(cosine_sim) > 0.4:
        print("  → Moderately aligned (some overlap)")
    else:
        print("  → Weakly aligned (different patterns)")

    print(f"\nPerformance Comparison:")
    print(f"  PCA (unsupervised):        AUC = {pca_auc:.3f}")
    print(f"  Supervised (LogReg):       AUC = {layer_comparison_results[best_layer]['auc']:.3f}")
    print(f"  Improvement:               {(layer_comparison_results[best_layer]['auc'] - pca_auc):.3f} ({(layer_comparison_results[best_layer]['auc'] - pca_auc)/pca_auc*100:+.1f}%)")

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Supervised distribution
ax = axes[0, 0]
safe_proj = supervised_projections[y_test == 1]
unsafe_proj = supervised_projections[y_test == 0]
bins = np.linspace(min(supervised_projections), max(supervised_projections), 30)
ax.hist(unsafe_proj, bins=bins, alpha=0.6, label='Unsafe', color='red', density=True)
ax.hist(safe_proj, bins=bins, alpha=0.6, label='Safe', color='green', density=True)
ax.set_xlabel('Projection Score', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title(f'Supervised Detection (AUC={layer_comparison_results[best_layer]["auc"]:.3f})', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: PCA distribution (if available)
ax = axes[0, 1]
if pca_available:
    safe_proj_pca = pca_projections[y_test == 1]
    unsafe_proj_pca = pca_projections[y_test == 0]
    bins_pca = np.linspace(min(pca_projections), max(pca_projections), 30)
    ax.hist(unsafe_proj_pca, bins=bins_pca, alpha=0.6, label='Unsafe', color='red', density=True)
    ax.hist(safe_proj_pca, bins=bins_pca, alpha=0.6, label='Safe', color='green', density=True)
    ax.set_xlabel('Projection Score', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'PCA Detection (AUC={pca_auc:.3f})', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'PCA results not available\nfor this layer',
            ha='center', va='center', fontsize=12, transform=ax.transAxes)
    ax.set_xticks([])
    ax.set_yticks([])

# Plot 3: ROC Curves comparison
ax = axes[1, 0]
fpr_sup, tpr_sup, _ = roc_curve(y_test, y_prob)
ax.plot(fpr_sup, tpr_sup, linewidth=2.5, color='darkorange',
        label=f'Supervised (AUC={layer_comparison_results[best_layer]["auc"]:.3f})')

if pca_available:
    fpr_pca, tpr_pca, _ = roc_curve(y_test, pca_projections)
    ax.plot(fpr_pca, tpr_pca, linewidth=2.5, color='steelblue',
            label=f'PCA (AUC={pca_auc:.3f})')
    ax.fill_between(fpr_pca, 0, tpr_pca, alpha=0.15, color='steelblue')

ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random')
ax.fill_between(fpr_sup, 0, tpr_sup, alpha=0.2, color='darkorange')
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title('ROC Curve Comparison', fontsize=12, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

# Plot 4: Performance summary
ax = axes[1, 1]
methods = ['Supervised']
aucs = [layer_comparison_results[best_layer]['auc']]
colors_bar = ['darkorange']

if pca_available:
    methods.insert(0, 'PCA')
    aucs.insert(0, pca_auc)
    colors_bar.insert(0, 'steelblue')

bars = ax.barh(methods, aucs, color=colors_bar, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, linewidth=1.5, label='Random baseline')
ax.set_xlabel('AUC Score', fontsize=11)
ax.set_title(f'Performance Summary (Layer {best_layer})', fontsize=12, fontweight='bold')
ax.set_xlim([0, 1])
ax.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for i, (bar, auc_val) in enumerate(zip(bars, aucs)):
    ax.text(auc_val + 0.02, i, f'{auc_val:.3f}', va='center', fontsize=11, fontweight='bold')

ax.legend()

plt.tight_layout()
plt.savefig('results/student_analysis/q6_supervised_vs_pca_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

### Required Analysis for Question 5

1. **Understanding Supervised Learning (2 points)** What is the key difference between the supervised approach (logistic regression) and
   the unsupervised approach (PCA) for safety detection? Consider:
   - What information each method uses
   - What each method optimizes for
   - When you would prefer one over the other


2. **Layer Analysis Results (3 points)** Based on your layer comparison experiments:
   - Which layer(s) achieved the best performance? (Report specific layer numbers and metrics)
   - Does performance improve, decline, or show a non-monotonic pattern?
   - What might explain this trend based on what you know about how neural networks process information hierarchically?


3. **Supervised vs Unsupervised Comparison (3 points)**
   - Compare the supervised direction with PCA's first principal component: What is the cosine similarity between them? Are they aligned or different? What does this tell you?
   - Compare the AUC scores between supervised and unsupervised: Which approach performs better for safety detection? By how much? Given that supervised learning requires labeled data, is the performance gain worth the cost?

## **Question 6**: Understanding the Data Efficiency Curve (15 points)

In the previous question, you saw that supervised learning (Logistic Regression) significantly outperforms  unsupervised learning (PCA). But there's a catch: **supervised learning requires labeled data**.

Getting high quality data can be costly. **More data = better performance**: But at what point do we hit diminishing returns?

Let's investigate this by training logistic regression with varying amounts of labeled data and compare against PCA baseline (which needs no labels).

In [ ]:
def supervision_learning_curve(cb_data, model, tokenizer,
                              n_samples=[5, 10, 20, 50],
                              n_runs=5):
    """
    Analyze how supervised learning performance improves with more labeled data.

    Question: How many labeled examples do we need before supervised
    learning outperforms unsupervised PCA? What's the point of
    diminishing returns?

    Strategy:
    1. Use safety dataset for consistent train/test split
    2. Train supervised detectors with varying amounts of training data
    3. Compare against PCA baseline (trained on full training set)
    4. Find the "sweet spot" for labeling effort

    Args:
        n_samples: List of sample sizes to test
        n_runs: Number of runs per sample size for variance estimation
    """
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.linear_model import LogisticRegression

    # Store results for each run
    all_results = {n: {'supervised': [], 'pca': []} for n in n_samples}

    layer = list(safety_hidden.keys())[-1]

    # Split safety dataset into train/test (80/20)
    all_H = safety_hidden[layer]
    all_labels = np.array([1 if l == 'safe' else 0 for l in safety_labels])

    # Use a fixed random state for reproducible train/test split
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        all_H, all_labels, test_size=0.2, random_state=42, stratify=all_labels
    )

    # Get PCA baseline (trained on full training set, evaluated on test set)
    components, mean, var_ratio = pca_from_scratch(X_train_full, n_components=1)
    pca_proj = project_onto_components(X_test, components, mean)[:, 0]
    if np.mean(pca_proj[y_test == 1]) < np.mean(pca_proj[y_test == 0]):
        pca_proj = -pca_proj

    pca_auc = roc_auc_score(y_test, pca_proj)

    # Run multiple trials for each sample size
    for n in n_samples:
        if n > len(X_train_full):
            continue

        # Need at least 2 samples per class for stratified sampling
        if n < 4:
            print(f"Skipping n={n}: need at least 4 samples for balanced sampling")
            continue

        for run in range(n_runs):
            # Stratified random sample from training set (ensures both classes present)
            # Sample n//2 from each class
            n_per_class = n // 2

            safe_indices = np.where(y_train_full == 1)[0]
            unsafe_indices = np.where(y_train_full == 0)[0]

            # Randomly sample from each class
            selected_safe = np.random.choice(safe_indices, n_per_class, replace=False)
            selected_unsafe = np.random.choice(unsafe_indices, n_per_class, replace=False)

            indices = np.concatenate([selected_safe, selected_unsafe])
            np.random.shuffle(indices)  # Shuffle to mix classes

            X_train_subset = X_train_full[indices]
            y_train_subset = y_train_full[indices]

            # Train logistic regression on subset
            clf = LogisticRegression(max_iter=1000, random_state=None)  # Allow randomness across runs
            clf.fit(X_train_subset, y_train_subset)

            # Evaluate on test set
            y_prob = clf.predict_proba(X_test)[:, 1]
            supervised_auc = roc_auc_score(y_test, y_prob)

            all_results[n]['supervised'].append(supervised_auc)
            all_results[n]['pca'].append(pca_auc)

    # Compute statistics
    results = []
    for n in n_samples:
        if n in all_results and all_results[n]['supervised']:
            sup_mean = np.mean(all_results[n]['supervised'])
            sup_std = np.std(all_results[n]['supervised'])
            results.append({
                'n_samples': n,
                'supervised_auc': sup_mean,
                'supervised_std': sup_std,
                'pca_auc': pca_auc,
                'improvement': (sup_mean - pca_auc) / pca_auc * 100,
            })

    return results

In [ ]:
# ============================================================
# TODO: Question 6 - Data Efficiency Experiment
# ============================================================

# Step 1: Define sample sizes to test
# Test how performance changes with different amounts of labeled data
# YOUR CODE HERE: Choose sample sizes to test
# Hint: Try a range like [10, 20, 50, 100, 200, 500]
sample_sizes_to_test = [10, 20, 50, 100, 200]  # Modify if needed

# Step 2: Run the analysis
# The supervision_learning_curve function is already implemented for you
# It will train classifiers with different amounts of data and plot results

# YOUR CODE HERE: Call the function with your sample sizes
# Hint: learning_curve_results = supervision_learning_curve(
#           cb_data=cb_data,
#           model=model,
#           tokenizer=tokenizer,
#           n_samples=sample_sizes_to_test,
#           n_runs=5  # Average over 5 runs for stability
#       )

learning_curve_results = supervision_learning_curve(cb_data, model, tokenizer, sample_sizes_to_test, 5)

# The function will automatically generate a learning curve plot below
# Examine the plot to answer the analysis questions

# Visualize learning curve
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Extract data
n_samples = [r['n_samples'] for r in learning_curve_results]
supervised_aucs = [r['supervised_auc'] for r in learning_curve_results]
supervised_stds = [r['supervised_std'] for r in learning_curve_results]
pca_aucs = [r['pca_auc'] for r in learning_curve_results]

# Plot: Learning curves with shaded uncertainty
ax.plot(n_samples, supervised_aucs, 'o-', label='Supervised (LogReg)',
        linewidth=2.5, markersize=10, color='darkorange')
ax.fill_between(n_samples,
                np.array(supervised_aucs) - np.array(supervised_stds),
                np.array(supervised_aucs) + np.array(supervised_stds),
                alpha=0.3, label='±1 std', color='darkorange')
ax.axhline(y=pca_aucs[0], color='steelblue', linestyle='--',
           linewidth=2.5, label=f'PCA Baseline (AUC={pca_aucs[0]:.3f})')
ax.set_xlabel('Number of Training Samples', fontsize=12)
ax.set_ylabel('AUC Score', fontsize=12)
ax.set_title('Data Efficiency: How Performance Scales with Labeled Data', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
ax.set_xticks(n_samples)
ax.set_xticklabels([str(n) for n in n_samples])
ax.set_ylim([0.5, 1.0])  # Set reasonable y-axis limits

plt.tight_layout()
plt.savefig('results/student_analysis/q7_supervision_learning_curve.png', dpi=150, bbox_inches='tight')
plt.show()

### Required Analysis for Question 6

Based on your learning curve results, answer the following:

**1. Crossover Analysis (2 points)**
- At what sample size does supervised learning reliably outperform PCA?
- What factors might influence this threshold? (Consider: data quality, task difficulty, feature dimensionality, class balance)

**2. Learning Curve Shape (2 points)**
- Describe the shape of the learning curve (linear, logarithmic, plateau?)
- Where do you observe diminishing returns? (Identify the "knee" in the curve)
- What does this suggest about data efficiency for this task?

**3. Critical Thinking on Problem Setup (2 points)**

Imagine you're a data scientist at a company deciding whether to purchase additional labeled data from a vendor or hire annotators. You want to use learning curves like the one above to make this decision.

**Your task:**
- What is unrealistic or problematic about using our current experimental setup to make real-world data acquisition decisions?
- Why might this approach fail or be misleading in practice?
- What would need to change to make data valuation more practical and reliable?

**Consider:**
- The timing of when you need to make the decision vs. when you can evaluate
- What happens if your modeling approach changes later
- What information you have access to at decision time vs. evaluation time

**Hint:** Think about the difference between evaluating something you already have versus deciding whether to acquire something you don't have yet. Papers like [LAVA: Data Valuation without Pre-Specified Learning Algorithms](https://arxiv.org/abs/2305.00054) tackle related challenges in data valuation.


# Conclusions

This neural archaeology project has taken you on a comprehensive journey through the internal representations of language models, revealing fundamental insights about how these systems organize and process information.
- **Layer-wise Information Processing** This hierarchical processing mirrors findings in neuroscience about cortical processing hierarchies, suggesting deep parallels between biological and artificial neural systems.

- **Concept Geometry and Dimensionality** The natural separability of important concepts without explicit training suggests that language models develop structured internal representations as an emergent property of next-token prediction.

- **Supervised vs. Unsupervised Learning Trade-offs** Our comparative analysis demonstrated critical insights about representation learning.

**Future Research Directions:** The path forward leads toward mechanistic interpretability, moving beyond correlations to causal understanding of actual circuits rather than just directional patterns. Robust alignment research must handle distribution shifts, resist adversarial attacks, and maintain capabilities while ensuring safety. Scalable oversight systems need to automate safety evaluation, learn efficiently from human feedback, and build self-improving safety mechanisms. Compositional control represents perhaps the most ambitious frontier: handling multiple objectives simultaneously, learning to compose interventions dynamically, and building truly modular safety systems. This journey marks not an end but a beginning, with the tools and insights gained serving as foundations for deeper exploration into the profound mysteries of artificial intelligence safety.

---

**END OF ASSIGNMENT**


## Acknowledgments

**Course Project**: This work was developed as an AI portfolio project for **ECE4424/CS4824 Machine Learning: Learn by Building** taught by Dr. Ming Jin at Virginia Tech (https://jinming99.github.io/learn-ml-by-building/)

**Teaching Assistants**: Special thanks to Kamal Sherawat (kamals@vt.edu) and Darshan Nere (darshannere@vt.edu) for their invaluable assistance in fine-tuning this notebook and generating promising results.

**Key References:**

- **Representation Engineering** - [Zou et al., 2023](https://arxiv.org/abs/2310.01405)
- **Circuit Breaker** - [Zou et al., 2024](https://www.circuit-breaker.ai/)
- **Backtracking for Safety** - [Zhang et al., 2024](https://arxiv.org/abs/2409.14586), [Sel et al., 2025](https://arxiv.org/abs/2503.08919)

**Datasets:**
- [Representation Engineering Github](https://github.com/andyzoujm/representation-engineering): emotion dataset
- [Circuit Breaker Github](https://github.com/GraySwanAI/circuit-breakers/tree/main/data): safety dataset

**AI-Assisted Development**: Large language models were utilized as collaborative tools in developing this educational material, assisting with code generation, documentation, and exploring different implementation strategies. All outputs were thoroughly reviewed, tested, and refined to ensure accuracy and alignment with course objectives.

**Ethical Note**: These techniques carry significant responsibility. Users should apply these methods ethically and exclusively for beneficial purposes, always considering the potential impacts of AI system modifications on end users and society.